In [14]:
import numpy as np
import scipy
from scipy import stats

from init import init_Y
from metrics import X_in_A, rho, eta, ESS, prop_alive_func
from stats import sampling_params, random_walk_on_params, pi_n, X_update, metropolis_X_theta

In [18]:
# Step 0 : initialisation
alpha = 0.9
N = 1000
Nt = 500
Y = init_Y()
Xs = [[1]]*N
epsilon_final = 0.00045
epsilon_t = 1000
espilon_list = [epsilon_t]
random_walk_stds = (0.05, 0.05, 0.7)
T = 10_000
Ws = 1/N * np.ones(N)
prop_alive = prop_alive_func(Ws)
thetas = np.array([sampling_params() for _ in range(N)])
Xs = [X_update(X, thetas[idx][0], thetas[idx][1],
               thetas[idx][2]) for idx, X in enumerate(Xs)]

In [12]:
t = 0
while epsilon_t > epsilon_final:
    #Step 1 : sampling
    print(f'Step 1 : t={t}')
    new_prop_alive = prop_alive
    while new_prop_alive > alpha*prop_alive:
        new_weights = Ws * np.array([int(X_in_A(Y, X, epsilon_t)) for X in Xs])
        new_prop_alive = prop_alive_func(new_weights)
        epsilon_t = 0.75 * epsilon_t # Decreasing of epsilon_t until the new_prop_alive is lower than alpha*prop_alive
    Ws = new_weights
    if epsilon_t < epsilon_final:
        epsilon_t = epsilon_final
    espilon_list.append(epsilon_t)

    # Step 2 : resampling
    print('Step 2')
    ESS_Ws = ESS(Ws)
    if ESS_Ws < Nt:
        indices = np.random.choice(range(N), size=N, replace=True, p=Ws) # TODO : corriger
        Xs = [Xs[i] for i in indices]
        Ws = 1/N * np.ones(N)
        thetas = np.array([sampling_params() for _ in range(N)])

    # Step 3 : random walk
    print('Step 3')
    for index, X in enumerate(Xs):
        if Ws[index] > 0:
            phi, tau, xi = thetas[index]
            X, phi, tau, xi= metropolis_X_theta(X, Y, phi, tau, xi,
                                                       epsilon_t, random_walk_stds)
            thetas[index] = (phi, tau, xi)
    print(Xs)
    t+=1
    print(f'espilon_t = {epsilon_t}, epsilon_final = {epsilon_final}')

Step 1 : t=0
Step 2
Step 3


c:\Users\33646\Desktop\ENSAE\HMCMC\metrics.py:13: RuntimeWarning: invalid value encountered in scalar divide
  eta_2 = 1 - np.sum([(genom_pop/total_pop)**2 for genom_pop in X])
c:\Users\33646\Desktop\ENSAE\HMCMC\metrics.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  result =  (1/n *


[[0], [2], [2], [2], [2], [0], [2], [0], [0, 1], [0], [2], [0, 1], [2], [0], [0], [2], [0], [2], [0], [2], [2], [2], [0], [0], [2], [0], [2], [2], [0], [2], [0], [0, 1], [2], [2], [0], [0], [2], [0], [2], [2], [2], [2], [2], [2], [0], [2], [0], [0], [2], [0, 1], [2], [2], [2], [0], [0, 1], [0], [2], [0], [2], [0], [0], [0, 1], [2], [2], [2], [2], [2], [2], [2], [2], [0], [2], [0], [2], [2], [2], [2], [0, 1], [0, 1], [0], [0], [2], [0], [0], [2], [2], [2], [2], [2], [2], [0], [2], [0], [2], [2], [2], [2], [2], [2], [2], [2], [2], [0], [2], [2], [0], [0], [2], [0], [2], [2], [0], [2], [2], [0], [2], [2], [2], [2], [0], [2], [0], [2], [0], [2], [2], [0, 1], [0], [0], [2], [2], [0], [0], [0], [0], [2], [2], [2], [2], [0], [2], [2], [0, 1], [2], [2], [0], [2], [2], [2], [2], [0], [0], [0], [2], [0], [0, 1], [2], [2], [2], [2], [0], [2], [2], [2], [2], [2], [0], [0], [2], [0], [0, 1], [0], [2], [2], [0], [0], [2], [2], [0], [2], [2], [2], [0], [2], [2], [0, 1], [2], [2], [0], [2], [0, 1], [2

KeyboardInterrupt: 

[[0],
 [0],
 [2],
 [2],
 [2],
 [0],
 [0],
 [2],
 [0],
 [0],
 [2],
 [2],
 [0],
 [2],
 [2],
 [2],
 [0],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [0],
 [0],
 [2],
 [0],
 [0],
 [0, 1],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [0],
 [2],
 [0],
 [2],
 [0],
 [0],
 [2],
 [0, 1],
 [2],
 [0],
 [0],
 [0],
 [2],
 [2],
 [0, 1],
 [0],
 [0],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [0],
 [0],
 [2],
 [0],
 [2],
 [0],
 [0],
 [0],
 [0],
 [0, 1],
 [2],
 [0],
 [0],
 [2],
 [2],
 [0, 1],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [2],
 [0, 1],
 [2],
 [2],
 [2],
 [0],
 [0],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [0],
 [2],
 [2],
 [2],
 [2],
 [0],
 [0],
 [0],
 [0],
 [2],
 [2],
 [0],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [2],
 [2],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [2],
 [0],
 [2],
 [2],
 [0, 1],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [0],
 [0],
 [2],
 [2],
 [0],
 [2],
 [0],
 [2],
 [2],
 [2],
 [0],
 [0],
 [0],
 [2],
 